## [model_consumer]kullm_polyglot_12_8b_in_context_learning_ml_g5_12xl

이 노트북에서는 한국어 모델을 활용하여 in-context learning을 통한 N-shot 학습을 달성하는 데 중점을 둡니다. 여기에는 모델의 자연어 이해(NLU) 기능을 활용하여 가상 어시스턴트 응답을 개인화하고 사용자를 위한 성능을 개선하는 것이 포함됩니다.

이 모듈에서는 고려대학교 [NLP & AI 연구실](http://blp.korea.ac.kr/)과 [HIAI 연구소](http://hiai.korea.ac.kr/)가 개발한 한국어 Large Language Model (LLM)인 KULLM (구름)을 기반으로 모델을 활용하고 있습니다.
구름 프로젝트는 한국어 모델 뿐만 아니라, 데이터 셋까지 전면 공개하여 한국어 LLM 생태계에 기여하고자 하였습니다.

<!-- <img src="../figures/flan-t5.png"  width="700" height="370"> -->

전반적으로 이 모듈은 **텍스트 요약, 추상적 질문 답변, 감정 분석, 감정 구문 추출**과 같은 NLU 작업을 해결하는 데 있어 FLAN-T5-XL의 기능을 살펴볼 수 있는 훌륭한 기회를 제공합니다.
[](https://github.com/nlpai-lab/KULLM)

In [2]:
# !pip install -U sagemaker

#### Imports 

In [3]:
from sagemaker.predictor import Predictor
from sagemaker import get_execution_role
from sagemaker.model import Model
from sagemaker import script_uris
from sagemaker import image_uris 
from sagemaker import model_uris
import sagemaker
import logging
import boto3
import time
import json
import pprint

sagemaker_session = sagemaker.Session()
region = sagemaker_session.boto_region_name
role = sagemaker.get_execution_role()

#### Setup essentials 

In [4]:
logger = logging.getLogger('sagemaker')
logger.setLevel(logging.DEBUG)
logger.addHandler(logging.StreamHandler())

In [5]:
logger.info(f'Using sagemaker=={sagemaker.__version__}')
logger.info(f'Using boto3=={boto3.__version__}')

Using sagemaker==2.165.0
Using boto3==1.26.155


In [6]:
# MODEL_ID = 'kullm-polyglot-5-8b-v2'  # the error is that "Expected (head_size % 8 == 0) && (head_size <= 128) to be true, but got false. "
MODEL_ID = 'kullm-polyglot-12-8b-v2'
MODEL_VERSION = '*'
INSTANCE_TYPE = 'ml.g5.12xlarge'
INSTANCE_COUNT = 1
IMAGE_SCOPE = 'inference'
MODEL_DATA_DOWNLOAD_TIMEOUT = 3600  # in seconds
CONTAINER_STARTUP_HEALTH_CHECK_TIMEOUT = 360
EBS_VOLUME_SIZE = 256  # in GB
CONTENT_TYPE = 'application/json'

# set up roles and clients 
client = boto3.client('sagemaker-runtime')
ROLE = get_execution_role()
logger.info(f'Role => {ROLE}')

Role => arn:aws:iam::322537213286:role/service-role/AmazonSageMaker-ExecutionRole-20230528T120509


#### I. Deploy Korean model as a SageMaker endpoint

이 모델은 SageMaker Deployment에서 [text-generation-inference](https://github.com/huggingface/text-generation-inference/tree/main)를 활용하여 Endpoint를 생성합니다. text-generation-inference 는 텍스트 생성 추론을 위한 Rust, Python 및 gRPC 서버이며, HuggingFace의 production에서 LLM의 API 추론 위젯을 구동하는 데 사용됩니다.

<img src="../figures/architecture.jpg" width="700" height="370">




[주요 특성]
- 간단한 launcher로 가장 인기 있는 대규모 언어 모델 제공
- 여러 GPU에서 더 빠른 추론을 위한 텐서 병렬 처리
- 서버 전송 이벤트(SSE, Server-Sent Events)를 사용한 토큰 스트리밍
- 총 처리량 증가를 위한 수신 요청의 지속적인 batching 처리
- 가장 많이 사용되는 아키텍처에 flash-attention을 사용하여 추론하도록 최적화된 transformers 코드
- bitsandbytes을 이용한 Quantization
- Safetensors 가중치 로딩
- 대규모 언어 모델용 워터마크를 사용한 워터마킹
- Logits 와퍼(temperature 스케일링, top-p, top-k, repetition penalty, 자세한 내용은 transformers.LogitsProcessor 참조)
- 시퀀스 중지
- 로그 확률
- 프로덕션 준비 완료(Open Telemetry, Prometheus metrics를 사용한 분산 추적)


In [7]:
from sagemaker.huggingface import HuggingFaceModel, get_huggingface_llm_image_uri

In [8]:
deploy_image_uri = get_huggingface_llm_image_uri(
  backend="huggingface", # or lmi
  region=region
)
deploy_image_uri

Defaulting to only available Python version: py39
Defaulting to only supported image scope: gpu.


'763104351884.dkr.ecr.us-west-2.amazonaws.com/huggingface-pytorch-tgi-inference:2.0.0-tgi0.8.2-gpu-py39-cu118-ubuntu20.04'

In [9]:
env = {
    # 'HF_MODEL_ID':'nlpai-lab/kullm-polyglot-5.8b-v2',  ## Expected (head_size % 8 == 0) && (head_size <= 128) to be true, but got false.  
    'HF_TASK':'text-generation',
    'HF_MODEL_ID':'nlpai-lab/kullm-polyglot-12.8b-v2',
    'SAGEMAKER_MODEL_SERVER_TIMEOUT': str(3600),
    'SM_NUM_GPUS':'4',  ## ml.g5.12xlarge 기준
    'HF_MODEL_QUANTIZE':'bitsandbytes',  ##[possible values: bitsandbytes, gptq]
}

In [10]:
unix_time = int(time.time())
endpoint_name = f'{MODEL_ID}-{unix_time}'
logger.info(f'Endpoint name: {endpoint_name}')

Endpoint name: kullm-polyglot-12-8b-v2-1687674586


In [11]:
model = HuggingFaceModel(
    image_uri=deploy_image_uri,
    env=env,
    role=ROLE,
    name=endpoint_name
)

In [12]:
%%time

_ = model.deploy(initial_instance_count=INSTANCE_COUNT, 
                 instance_type=INSTANCE_TYPE, 
                 endpoint_name=endpoint_name, 
                 # volume_size=EBS_VOLUME_SIZE, 
                 model_data_download_timeout=MODEL_DATA_DOWNLOAD_TIMEOUT, 
                 container_startup_health_check_timeout=CONTAINER_STARTUP_HEALTH_CHECK_TIMEOUT)

Creating model with name: kullm-polyglot-12-8b-v2-1687674586
CreateModel request: {
    "ModelName": "kullm-polyglot-12-8b-v2-1687674586",
    "ExecutionRoleArn": "arn:aws:iam::322537213286:role/service-role/AmazonSageMaker-ExecutionRole-20230528T120509",
    "PrimaryContainer": {
        "Image": "763104351884.dkr.ecr.us-west-2.amazonaws.com/huggingface-pytorch-tgi-inference:2.0.0-tgi0.8.2-gpu-py39-cu118-ubuntu20.04",
        "Environment": {
            "HF_TASK": "text-generation",
            "HF_MODEL_ID": "nlpai-lab/kullm-polyglot-12.8b-v2",
            "SAGEMAKER_MODEL_SERVER_TIMEOUT": "3600",
            "SM_NUM_GPUS": "4",
            "HF_MODEL_QUANTIZE": "bitsandbytes",
            "SAGEMAKER_PROGRAM": "",
            "SAGEMAKER_SUBMIT_DIRECTORY": "",
            "SAGEMAKER_CONTAINER_LOG_LEVEL": "20",
            "SAGEMAKER_REGION": "us-west-2"
        }
    }
}
Creating endpoint-config with name kullm-polyglot-12-8b-v2-1687674586
Creating endpoint with name kullm-polyglot-12

---------------!CPU times: user 182 ms, sys: 15.9 ms, total: 198 ms
Wall time: 8min 3s


#### II. [테스트] SageMaker 엔드포인트를 호출하여 자연어 이해(NLU) 및 자연어 생성(NLG) 작업을 위한 배포된 모델 테스트

In [48]:
from utils.prompter import Prompter
prompter = Prompter("kullm")

def infer(instruction, input_text):
    prompt = prompter.generate_prompt(instruction, input_text)
    payload = {'inputs': prompt,
           'stream' : False,
           'parameters' : {
               'best_of' : 2,
               # 'details' : True,
               'do_sample' : True,
               'max_new_tokens': 614, 
               'return_full_text': True,
               'repetition_penalty' : 1.5,
               'temperature': 0.7}
          }
    payload = json.dumps(payload).encode('utf-8')
    return payload

In [60]:
input_text = """
고객님: 안녕하세요, iPhone에 문제가 있습니다.
상담원: 안녕하세요! 무슨 문제인가요?
고객님: 휴대폰이 제대로 충전되지 않고 배터리가 매우 빨리 소모되는 것 같습니다. 다른 충전 케이블과 전원 어댑터를 사용해 보았지만 문제가 지속됩니다.
상담원: 흠, 그렇군요. 몇 가지 문제 해결 단계를 시도해 보겠습니다. 설정, 배터리로 이동하여 배터리 수명을 많이 소모하는 앱이 있는지 확인해 주시겠어요?
고객님: 예, 배터리를 많이 소모하는 앱이 몇 개 있습니다.
상담원: 좋아요, 화면 하단에서 위로 스와이프하여 해당 앱을 강제 종료한 다음 앱을 위로 스와이프하여 닫아 보세요.
고객: 그렇게 했는데도 문제가 여전히 남아 있습니다.
상담원: 네, iPhone의 설정을 기본값으로 재설정해 보겠습니다. 이렇게 해도 데이터가 삭제되지는 않습니다. 설정, 일반, 재설정으로 이동한 다음 모든 설정 재설정을 선택하세요.
고객님: 그렇게 했습니다. 다음은 어떻게 해야 하나요?
상담원: 이제 iPhone을 재시동해 보겠습니다. "밀어서 전원 끄기" 옵션이 표시될 때까지 전원 버튼을 길게 누릅니다. 밀어 전원을 끄고 몇 초간 기다린 다음 iPhone을 다시 켜세요.
고객님: 다시 시작했지만 여전히 제대로 충전되지 않습니다.
상담원: 그렇군요. iPhone에서 진단 테스트를 실행해야 할 것 같습니다. 가까운 Apple Store 또는 공인 서비스 제공업체를 방문하여 iPhone을 점검받으시기 바랍니다.
고객: 예약을 해야 하나요?
상담원: 예. 줄을 서서 기다리지 않으려면 항상 미리 예약하는 것이 가장 좋습니다. 온라인으로 예약하거나 Apple Store 또는 공인 서비스 제공업체에 전화하여 예약할 수 있습니다.
고객님: 수리 비용은 제가 지불해야 하나요?
상담원: iPhone에 보증이 적용되는지 여부에 따라 다릅니다. 보증이 적용되는 경우에는 비용을 지불할 필요가 없습니다. 그러나 보증이 적용되지 않는 경우에는 수리 비용을 지불하셔야 합니다.
고객님: iPhone을 돌려받는 데 얼마나 걸리나요?
상담원: 문제의 심각도에 따라 다르지만 일반적으로 영업일 기준 1~2일이 소요됩니다.
고객: 온라인으로 수리 상태를 추적할 수 있나요?
상담원: 온라인 또는 Apple Store 또는 공인 서비스 제공업체에 전화하여 수리 상태를 추적할 수 있습니다.
고객: 알겠습니다. 도와주셔서 감사합니다.
"""

Generation configuration 

endpoint를 호출할 때 이 텍스트를 JSON 페이로드 내에 제공해야 합니다. 이 JSON 페이로드에는 length, sampling strategy, output token sequence restrictions을 제어하는 데 도움이 되는 원하는 추론 매개변수가 포함될 수 있습니다. transformers library에는 [사용 가능한 페이로드 매개변수](https://huggingface.co/docs/transformers/main_classes/text_generation#transformers.GenerationConfig)의 전체 목록이 정의되어 있지만, 중요한 페이로드 매개변수는 다음과 같이 정의되어 있습니다:

* **do_sample (`bool`)** – logits sampling 활성화
* **max_new_tokens (`int`)** – 생성된 토큰의 최대 수
* **best_of (`int`)** – best_of 개의 시퀀스를 생성하고 가장 높은 토큰 로그 확률이 있는 경우 반환
* **repetition_penalty (`float`)** – 반복 패널티에 대한 파라미터, 1.0은 패널티가 없음을 의미하여 Greedy 서치와 동일, 커질수록 다양한 결과를 얻을 수 있으며, 자세한 사항은 [this paper](https://arxiv.org/pdf/1909.05858.pdf)을 참고
* **return_full_text (`bool`)** – 생성된 텍스트에 프롬프트를 추가할지 여부
* **seed (`int`)** – Random sampling seed
* **stop_sequences (`List[str]`)** – `stop_sequences` 가 생성되면 토큰 생성을 중지
* **temperature (`float`)** – logits 분포 모듈화에 사용되는 값
* **top_k (`int`)** – 상위 K개 만큼 가장 높은 확률 어휘 토큰의 수
* **top_p (`float`)** – 1 보다 작게 설정하게 되며, 상위부터 정렬했을 때 가능한 토큰들의 확률을 합산하여 `top_p` 이상의 가장 작은 집합을 유지
* **truncate (`int`)** – 입력 토큰을 지정된 크기로 잘라냄
* **typical_p (`float`)** – typical Decoding 양으로, 자세한 사항은 [Typical Decoding for Natural Language Generation](https://arxiv.org/abs/2202.00666)을 참고
* **watermark (`bool`)** –  [A Watermark for Large Language Models](https://arxiv.org/abs/2301.10226)가 Watermarking
* **decoder_input_details (`bool`)** – decoder input token logprobs와 ids를 반환


#### A. Text Summarization 

In [49]:
instruction = "다음 대화를 요약해 주세요"

payload = infer(instruction, input_text)

In [50]:
response = client.invoke_endpoint(EndpointName=endpoint_name, 
                                  ContentType=CONTENT_TYPE, 
                                  Body=payload)

model_predictions = json.loads(response['Body'].read().decode())
s = model_predictions[0]['generated_text']
generated_text = prompter.get_response(s)
pprint.pprint(f'Response: {generated_text}')

('Response: A) 고객에게 기기 및 연결된 액세서리(예 : 충전기 등), 저장 중인 개인 정보나 민감정보 보호 조치 등 발생 '
 '가능성이 높다고 생각 되면 중요하고 필수적 인 연락처 세부 사항 포함 B')


#### B. Abstractive Question Answering 

##### Q1

In [51]:
instruction = 'iPhone 충전 문제를 해결하기 위해 고객에게 어떤 문제 해결 단계를 제안했나요?'

payload = infer(instruction, input_text)

In [52]:
response = client.invoke_endpoint(EndpointName=endpoint_name, 
                                  ContentType=CONTENT_TYPE, 
                                  Body=payload)

model_predictions = json.loads(response['Body'].read().decode())
s = model_predictions[0]['generated_text']
generated_text = prompter.get_response(s)
pprint.pprint(f'Response: {generated_text}')

('Response: A/S 센터 직원으로서 저자는 상황 분석 결과 및 기기 사양 정보 등 다양하고 상세한 진술서 내용 일부만 공유했으며 '
 '특정 지침이나 해결책보다는 프로세스 중 발생 가능성이 높다고 판단된 이벤트나 사건 위주였으므로 이 게시물 섹션(10점)보다 더 많아야 함 '
 '--+--')


Q2

In [61]:
instruction = 'iPhone을 기본 설정으로 재설정하면 충전 문제와 배터리 소모 문제를 해결할 수 있나요?'

payload = infer(instruction, input_text)

In [62]:
response = client.invoke_endpoint(EndpointName=endpoint_name, 
                                  ContentType=CONTENT_TYPE, 
                                  Body=payload)

model_predictions = json.loads(response['Body'].read().decode())
s = model_predictions[0]['generated_text']
generated_text = prompter.get_response(s)
pprint.pprint(f'Response: {generated_text}')

'Response: 예'


Q3

In [68]:
instruction = '고객이 iPhone 수리를 위해 가까운 Apple Store 또는 공인 서비스 제공업체에 예약하려면 어떤 조치를 취해야 하나요?'

payload = infer(instruction, input_text)

In [ ]:
response = client.invoke_endpoint(EndpointName=endpoint_name, 
                                  ContentType=CONTENT_TYPE, 
                                  Body=payload)

model_predictions = json.loads(response['Body'].read().decode())
s = model_predictions[0]['generated_text']
generated_text = prompter.get_response(s)
pprint.pprint(f'Response: {generated_text}')

('Response: According to the instructions provided by a representative of an '
 'external service Promissory Serveices Purchase Registry (PSR) department '
 'suitable for customers with mobile team II-Main(iOS), you would follow this '
 'generously')


#### C. Sentiment Analysis

In [65]:
instruction = '고객과 상담원 간의 대화에 대한 전반적인 감정 및 감정 점수는 얼마인가요?'

payload = infer(instruction, input_text)

In [66]:
response = client.invoke_endpoint(EndpointName=endpoint_name, 
                                  ContentType=CONTENT_TYPE, 
                                  Body=payload)

model_predictions = json.loads(response['Body'].read().decode())
s = model_predictions[0]['generated_text']
generated_text = prompter.get_response(s)
pprint.pprint(f'Response: {generated_text}')

'Response: 점수 || ---- |------------'


In [45]:
%store endpoint_name

Stored 'endpoint_name' (str)


In [58]:
# # Delete the SageMaker endpoint
# _.delete_model()
# _.delete_endpoint()

Deleting model with name: huggingface-text2text-flan-t5-xl-1685337480
Deleting endpoint configuration with name: huggingface-text2text-flan-t5-xl-1685337480
Deleting endpoint with name: huggingface-text2text-flan-t5-xl-1685337480
